# Gradient Descent

## Ideas
* continuous functions / discontinuous functions
* 1d function / 2d function
* let users do a step function
* generate animations of their stepping with plotly
    - surface in 3d + arrow + projected contour plot + projected arrow
    
* user missions
    - define numerical symmetric derivative using given x and h.
    - define gradient descent step function


# Mission - 3d graph + contour plot + arrows

In [147]:
import math
import numpy as np
from plotly import tools
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)

h = 0.001
gamma = 0.2 # step size multiplier
precision = 0.001
step_size = math.inf

def f(x, y):
    return np.sin((x/2-1.25)**2 + (y/2-1.25)**2) * np.cos(x + np.exp(-y))
    #return 2 * np.sin(- 1/4 * x**2 + 1/4 * (y-2)**2 + 3) * np.cos(x + np.exp(-y+1))

def dfx(x, y):
    """Meant to be defined by the user..."""
    return (f(x+h,y) - f(x-h,y)) / (2*h)
def dfy(x, y):
    """Meant to be defined by the user..."""
    return (f(x,y+h) - f(x,y-h)) / (2*h)

def gradient_descent_step(x, y, gamma):
    """Meant to be defined by the user...
    returns the step size taken and the the new x"""
    prev_x, prev_y = x, y
    x, y = prev_x - gamma * dfx(prev_x, prev_y), prev_y - gamma * dfy(prev_x, prev_y)
    step_size = np.linalg.norm((x - prev_x, y - prev_y))
    return x, y, step_size

# Jupyter Notebook responsive sizes - Small: 618, Medium: 790, Large: 990
layout = go.Layout(
    width=618, height=618,
    margin=dict(l=40, r=40, b=40, t=80),
    scene=dict(
        camera=dict(
            up=dict(x=1, y=0, z=0),
            center=dict(x=0, y=0, z=0),
            eye=dict(x=0, y=0, z=2)
        )
    )
)

# Gradient Descent
x, y = 1, 1.9 # starting position

steps = 0
x_steps, y_steps, z_steps = [x], [y], [f(x,y)]
while step_size > precision and steps < 1000:
    x, y, step_size = gradient_descent_step(x, y, gamma)
    x_steps.append(x)
    y_steps.append(y)
    z_steps.append(f(x,y))
    steps += 1 

print("The local minimum occurs at ({:.2}, {:.2}) after {} steps".format(x, y, len(z_steps)))

# Main surface plot
nx, ny = (30, 30)
x_0, x_1 = (0, 5)
y_0, y_1 = (0, 5)
x = np.linspace(x_0, x_1, nx)
y = np.linspace(y_0, y_1, ny)

xx, yy = np.meshgrid(x, y, sparse=True)
zz = f(xx, yy)

colorscale = 'Picnic'
surf = go.Surface(x=x, y=y, z=zz, showscale=False, colorscale=colorscale)
contour = go.Contour(x=x, y=y, z=zz, showscale=False, colorscale=colorscale, ncontours=10)
lines_contour = go.Scatter(x=x_steps, y=y_steps, mode='markers+lines', line=dict(color='black'))

dzx, dzy = np.gradient(zz, axis=(1,0))
# Create quiver figure

xxx, yyy = np.meshgrid(x,y)

fig_quiver = ff.create_quiver(xxx, yyy, dzx, dzy, scale=1, arrow_scale=.4, line=dict(width=1))
fig_quiver['layout'] = layout

lines_surf = go.Scatter3d(x=x_steps, y=y_steps, z=np.array(z_steps)+0.05,
                      mode='markers+lines', marker=dict(size=3), line=dict(color='black'),
)


fig_3d = go.Figure(data=[surf, lines_surf], layout=layout)
fig_3d['layout']['title'] = "2D-function's plot"

fig_quiver['data'].append(contour)
fig_quiver['data'].append(lines_contour)
#fig_contour = go.Figure(data=[contour, lines_contour], layout=layout)
fig_quiver['layout']['title'] = "2D-function's contour plot"

py.iplot(fig_quiver, show_link=False)
py.iplot(fig_3d, show_link=False)

The local minimum occurs at (2.2, 0.019) after 55 steps
